In [1]:
from decouple import config
import requests

In [16]:
assert config("POLOGYON_API_KEY", default=None, cast=str) is not None

In [ ]:
ticker = "AAPL"
multiplier = "5"
timespan = "minute"
from_date = "2023-01-09"
to_date = "2023-01-09"
path = f"/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
url = f"https://api.polygon.io{path}?apiKey={POLOGYON_API_KEY}"

In [17]:
import pytz
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode
from datetime import datetime


POLOGYON_API_KEY = config("POLOGYON_API_KEY", default=None, cast=str)


def transform_polygon_result(result):
    unix_timestamp = result.get("t") / 1000.0
    utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone("UTC"))
    return {
        "open_price": result.get("o"),
        "high_price": result.get("h"),
        "low_price": result.get("l"),
        "close_price": result.get("c"),
        "number_of_trades": result.get("n"),
        "volume": result.get("v"),
        "volume_weighted_average": result.get("vw"),
        "time": utc_timestamp,
    }


@dataclass
class PolygonAPIClient:
    ticker: str = "AAPL"
    multiplier: int = 5
    timespan: str = "minute"
    from_date: str = "2023-01-09"
    to_date: str = "2023-01-09"
    api_key: str = ""
    adjusted: bool = True
    sort: Literal["asc", "desc"] = "asc"

    def get_api_key(self):
        return self.api_key or POLOGYON_API_KEY

    def get_headers(self):
        api_key = self.get_api_key()
        return {"Authorization": f"Bearer {api_key}"}

    def get_params(self):
        return {"adjusted": self.adjusted, "sort": self.sort}

    def generate_url(self, pass_auth=False):
        path = f"/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}"
        url = f"https://api.polygon.io{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        results = data["results"]
        dataset = []
        for result in results:
            dataset.append(transform_polygon_result(result))
        return dataset

In [ ]:
stock_api_client = PolygonAPIClient(ticker="GOOG", multiplier=1)
stock_api_client.generate_url()

In [ ]:
dataset = stock_api_client.get_stock_data()
dataset